In [1]:
#Import required packages
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np

import datetime
import os

In [2]:
##### Input Database Credentials

localhost='pittsburghdb.c9rczggk5uzn.eu-west-2.rds.amazonaws.com'
username='pittsburghadmin'
password='Pitts$8burgh'
databasename='pittsburgh'
port=3306


FUNCTIONS

In [3]:
#Create a connection to database server
def create_server_connection(host, user, passwd):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user,
            passwd=passwd
        )
        print('Connected to server successfully')
    except Error as err:
        print(f'An error has been encountered!: "{err}"')

    return connection

In [4]:
#create a database
def create_database(connection, databasename):
    cursor = connection.cursor()
    try:
        cursor.execute('CREATE DATABASE {}'.format(databasename)) 
        print('Database created successfully')
    except Error as err:
        print(f'An error has been encountered!: "{err}"')

In [5]:
#create a database connection
def create_db_connection(host, user, passwd, database):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user,
            passwd=passwd,
            database=database
        )
        print('Connection to database '+database+' successful')
    except Error as err:
        print(f'An error has been encountered!: "{err}"')
        
    return connection



In [6]:
#run sql query on database with open connection
def run_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print('Query successfully executed')
    except Error as err:
        print(f'An error has been encountered!: "{err}"')

In [7]:
#Test Connection

connection = create_server_connection(localhost, username, password)

Connected to server successfully


In [ ]:
#Create database with variable inputs
create_database(connection,databasename)

CREATE TABLES

In [ ]:
#create stations table
create_stations_table = """
CREATE TABLE stations (  
  stationid INT(11) NOT NULL,
  stationname VARCHAR(255) NOT NULL,
  racksize INT(11) NOT NULL,
  latitude DECIMAL(12,8) NOT NULL,
  longitude DECIMAL(12,8) NOT NULL,
  PRIMARY KEY (stationid)
   );
 """

connection = create_db_connection(localhost, username, password, databasename) 

run_query(connection, create_stations_table) 



In [ ]:
#create journeys table
create_journeys_table = """
CREATE TABLE journeys (
  journeyid INT AUTO_INCREMENT,
  bikeid INT(11) NOT NULL,
  stationoutid INT(11) NOT NULL,
  datetimeout DATETIME NOT NULL,
  stationinid INT(11) NOT NULL,
  datetimein DATETIME NOT NULL,
  PRIMARY KEY (journeyid),
  FOREIGN KEY (stationoutid) REFERENCES stations(stationid),
  FOREIGN KEY (stationinid) REFERENCES stations(stationid)
  );
 """

connection = create_db_connection(localhost, username, password, databasename) 

run_query(connection, create_journeys_table) 

In [9]:
#create bikes out table

create_bikesout_table = """
CREATE TABLE bikesout (  
  bikeid INT(11) NOT NULL,
  stationid INT(11) NOT NULL,
  datetimeout DATETIME NOT NULL,
  PRIMARY KEY (bikeid),
  FOREIGN KEY (stationid) REFERENCES stations(stationid)
  );
 """

connection = create_db_connection(localhost, username, password, databasename) 

run_query(connection, create_bikesout_table) 

Connection to database pittsburgh successful
Query successfully executed


In [ ]:
#create
create_bikeslocations_table = """
CREATE TABLE bikeslocations (  
  bikeid INT(11) NOT NULL,
  stationid INT(11) NOT NULL,
  PRIMARY KEY (bikeid),
  FOREIGN KEY (stationid) REFERENCES stations(stationid)  
  );
 """

connection = create_db_connection(localhost, username, password, databasename) 

run_query(connection, create_bikeslocations_table) 

In [ ]:
#Import Pittsburgh Bike Station Information

df=pd.read_csv('PittsburghStations.csv')

connection = create_db_connection(localhost, username, password, databasename) 
cursor = connection.cursor()

for row in df.itertuples():
    cursor.execute('''
                    INSERT INTO stations (stationid, stationname, racksize, latitude, longitude)
                    VALUES (%s,%s,%s,%s,%s)
                    ''',
                    [row.StationID, 
                    row.StationName,
                    row.RackSize,
                    row.Latitude,
                    row.Longitude]
                    )
    connection.commit()

In [ ]:
#Import Pittsburgh Bike Journey Information

df=pd.read_csv('PittsburghJourneyData.csv')



connection = create_db_connection(localhost, username, password, databasename) 
cursor = connection.cursor()

for row in df.itertuples():
    cursor.execute('''                
                INSERT INTO journeys (bikeid, stationoutid, datetimeout, stationinid, datetimein)
                VALUES (%s, %s, %s, %s, %s)
                ''',
                [row.BikeID, 
                row.StationOutID,
                row.DateTimeOut,
                row.StationInID,
                row.DateTimeIn]
                )
connection.commit()

CLOSE DATABASE CONNECTION

In [ ]:
#close database connection
connection.close() 